In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import numpy as np
import math
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/Project/Steel_industry_data.csv')

In [52]:
#Rename some columns
df= df.rename(columns={'Lagging_Current_Reactive.Power_kVarh': 'Lagging_Reactive_Power_kVarh',
                        'Leading_Current_Reactive_Power_kVarh': 'Leading_Reactive_Power_kVarh',
                        'Lagging_Current_Power_Factor': 'Lagging_Power_Factor',
                        'Leading_Current_Power_Factor': 'Leading_Power_Factor',
                        'CO2(tCO2)':'CO2'})

In [53]:
#indexing date
df = df.set_index('date')
df

,Usage_kWh,Lagging_Reactive_Power_kVarh,Leading_Reactive_Power_kVarh,CO2,Lagging_Power_Factor,Leading_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
date,,,,,,,,,,
01/01/2018 00:15,3.17,2.95,0.00,0.0,73.21,100.00,900,Weekday,Monday,Light_Load
01/01/2018 00:30,4.00,4.46,0.00,0.0,66.77,100.00,1800,Weekday,Monday,Light_Load
01/01/2018 00:45,3.24,3.28,0.00,0.0,70.28,100.00,2700,Weekday,Monday,Light_Load
01/01/2018 01:00,3.31,3.56,0.00,0.0,68.09,100.00,3600,Weekday,Monday,Light_Load
01/01/2018 01:15,3.82,4.50,0.00,0.0,64.72,100.00,4500,Weekday,Monday,Light_Load
...,...,...,...,...,...,...,...,...,...,...
31/12/2018 23:00,3.85,4.86,0.00,0.0,62.10,100.00,82800,Weekday,Monday,Light_Load
31/12/2018 23:15,3.74,3.74,0.00,0.0,70.71,100.00,83700,Weekday,Monday,Light_Load
31/12/2018 23:30,3.78,3.17,0.07,0.0,76.62,99.98,84600,Weekday,Monday,Light_Load


In [54]:
# Map the values in the WeekStatus column
week_status_map = {'Weekday': 0, 'Weekend': 1}
df['WeekStatus'] = df['WeekStatus'].map(week_status_map)
# Map the values in the Day_of_week column
day_of_week_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
df['Day_of_week'] = df['Day_of_week'].map(day_of_week_map)
# Map the values in the Load_Type column
load_type_map = {'Light_Load': 0, 'Medium_Load': 1, 'Maximum_Load': 2}
df['Load_Type'] = df['Load_Type'].map(load_type_map)
df

,Usage_kWh,Lagging_Reactive_Power_kVarh,Leading_Reactive_Power_kVarh,CO2,Lagging_Power_Factor,Leading_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
date,,,,,,,,,,
01/01/2018 00:15,3.17,2.95,0.00,0.0,73.21,100.00,900,0,0,0
01/01/2018 00:30,4.00,4.46,0.00,0.0,66.77,100.00,1800,0,0,0
01/01/2018 00:45,3.24,3.28,0.00,0.0,70.28,100.00,2700,0,0,0
01/01/2018 01:00,3.31,3.56,0.00,0.0,68.09,100.00,3600,0,0,0
01/01/2018 01:15,3.82,4.50,0.00,0.0,64.72,100.00,4500,0,0,0
...,...,...,...,...,...,...,...,...,...,...
31/12/2018 23:00,3.85,4.86,0.00,0.0,62.10,100.00,82800,0,0,0
31/12/2018 23:15,3.74,3.74,0.00,0.0,70.71,100.00,83700,0,0,0
31/12/2018 23:30,3.78,3.17,0.07,0.0,76.62,99.98,84600,0,0,0


In [55]:
from sklearn.model_selection import train_test_split
X = df.drop('Usage_kWh', axis = 1)
y = df['Usage_kWh']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [56]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
y_pred = rf.predict(X_test)

In [57]:
# Save the model using Pickle
import pickle
filename = 'model.pkl'
pickle.dump(rf, open(filename, 'wb'))